<a href="https://colab.research.google.com/github/Pradoshks/COVID-19/blob/master/DLH_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
from google.colab import drive
import pandas as pd 
from torchsummary import summary

import torch
drive.mount('/content/drive')
from gensim.models import Word2Vec
#path = '/content/drive/MyDrive/Data/NOTEEVENTS.csv'
#df = pd.read_csv(path)
#annotations_path = "/content/drive/MyDrive/Data/annotations.csv"
#annotations = pd.read_csv(annotations_path)
#hamd = pd.merge(annotations, df, how = 'inner', left_on = ['Hospital.Admission.ID', 'subject.id'], right_on = ['HADM_ID', 'SUBJECT_ID'])
#discharges = hamd[(hamd.CATEGORY == 'Discharge summary') & (hamd.DESCRIPTION == 'Report')]#['DESCRIPTION'].unique()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#x = discharges[['Obesity', 'Non.Adherence', 'Developmental.Delay.Retardation',
        #'Advanced.Heart.Disease', 'Advanced.Lung.Disease',
        #'Schizophrenia.and.other.Psychiatric.Disorders', 'Alcohol.Abuse',
        #'Other.Substance.Abuse', 'Chronic.Pain.Fibromyalgia',
        #'Chronic.Neurological.Dystrophies', 'Advanced.Cancer', 'Depression',
        #'Dementia', 'Unsure', 'TEXT']]
#x.drop_duplicates(inplace = True)
#x.drop_duplicates(subset = ['TEXT'], keep = 'first', inplace = True)


In [ ]:
#x.to_csv('/content/drive/MyDrive/Data/final_file.csv')

In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
print(device)

There are 1 GPU(s) available.
Device name: Tesla T4
cuda


In [ ]:
dataframe = pd.read_csv('/content/drive/MyDrive/Data/final_file.csv')
dataframe.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import re 
import numpy as np 

class HealthcareTextDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.text_data = self.data['TEXT']
        self.labels = torch.tensor(np.array(self.data.iloc[:, :-1]), dtype = torch.float)  # assuming the labels are in the first 14 columns
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.text_data.iloc[idx]
        labels = self.labels[idx]
        sample = {'text': text, 'labels': labels}

        if self.transform:
            sample = self.transform(sample)
        return sample

In [ ]:
def word2idx(texts):
    word_to_idx = {'<pad>':0}
    idx = 1
    for i in range(len(texts)):
        for word in texts[i]['text']:
            if word not in word_to_idx:
                word_to_idx[word] = idx
                idx += 1
    print(word_to_idx)
    return word_to_idx

In [ ]:
tokenizer = get_tokenizer('basic_english')
import string
# Define a custom data transformation function
def transform_fn(sample):
    text = sample['text']
    labels = sample['labels']

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Tokenize text data
    text = tokenizer(text)

    # Convert labels to PyTorch tensor
    return {'text': text, 'labels': labels}

  
def collate_fn(batch):
    texts = [item['text'] for item in batch]  # Extract the text data from the batch
    labels = [item['labels'] for item in batch]  # Extract the labels from the batch

    # Find the maximum length of text sequences in the batch
    max_len = max([len(text) for text in texts])

    # Pad the text sequences with a padding token (e.g., '<pad>') to the maximum length
    padded_texts = []
    for text in texts:
        padded_text = text + ['<pad>'] * (max_len - len(text))
        padded_texts.append(padded_text)

    # Convert the labels to a PyTorch tensor
    labels = torch.stack(labels)

    return {'text': padded_texts, 'labels': labels}

In [ ]:
dataset = HealthcareTextDataset(dataframe, transform_fn)

In [ ]:
from torch.utils.data import Dataset, DataLoader


In [ ]:
# testing collate_fn 
test_func_loader = DataLoader(dataset, batch_size=10, shuffle=True, collate_fn=collate_fn)
for batch in test_func_loader:
  print(batch)
  break

{'text': [['admission', 'date', 'discharge', 'date', 'date', 'of', 'birth', 'sex', 'm', 'service', 'medicine', 'allergies', 'sulfonamides', 'attendingfirst', 'name', 'lf', 'chief', 'complaint', 'shortness', 'of', 'breath', 'major', 'surgical', 'or', 'invasive', 'procedure', 'picc', 'line', 'placement', 'thoracentesis', 'chest', 'tube', 'placement', 'history', 'of', 'present', 'illness', 'year', 'old', 'man', 'with', 'history', 'of', 'right', 'nonsmall', 'cell', 'lung', 'ca', 'sp', 'pneumonetctomy', 'copd', 'hypertension', 'sick', 'sinus', 'syndrome', 'sp', 'first', 'name', 'lf', 'and', 'icd', 'placement', 'bph', 'and', 'depression', 'presenting', 'with', 'shortness', 'of', 'breath', 'the', 'patient', 'had', 'been', 'admitted', 'with', 'dyspnea', 'extensive', 'work', 'up', 'at', 'that', 'time', 'included', 'cardiac', 'catheterization', 'which', 'showed', 'total', 'occlusion', 'of', 'rca', 'although', 'no', 'intervention', 'was', 'done', 'at', 'that', 'time', 'it', 'was', 'felt', 'his', 

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [ ]:
vocab = word2idx(dataset)
vocab_size = len(vocab)
print(vocab_size)

{'<pad>': 0, 'admission': 1, 'date': 2, 'discharge': 3, 'of': 4, 'birth': 5, 'sex': 6, 'f': 7, 'service': 8, 'cardiac': 9, 'intensive': 10, 'care': 11, 'medicine': 12, 'chief': 13, 'complaint': 14, 'the': 15, 'patient': 16, 'was': 17, 'admitted': 18, 'to': 19, 'unit': 20, 'on': 21, 'with': 22, 'acute': 23, 'myocardial': 24, 'infarction': 25, 'and': 26, 'fever': 27, 'history': 28, 'present': 29, 'illness': 30, 'is': 31, 'a': 32, 'year': 33, 'old': 34, 'white': 35, 'female': 36, 'coronary': 37, 'artery': 38, 'disease': 39, 'hypertension': 40, 'hypercholesterolemia': 41, 'two': 42, 'pack': 43, 'per': 44, 'day': 45, 'tobacco': 46, 'use': 47, 'previous': 48, 'bypass': 49, 'graft': 50, 'surgery': 51, 'presenting': 52, 'an': 53, 'outside': 54, 'hospital': 55, 'fevers': 56, 'confusion': 57, 'had': 58, 'ct': 59, 'scan': 60, 'chest': 61, 'at': 62, 'that': 63, 'time': 64, 'which': 65, 'revealed': 66, 'pneumonia': 67, 'by': 68, 'report': 69, 'in': 70, 'left': 71, 'lower': 72, 'lobe': 73, 'while': 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_filters, filter_sizes, output_size, dictionary):
        super(CNNClassifier, self).__init__()
        self.dictionary = dictionary
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv1d(embedding_dim, num_filters, fs) for fs in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * num_filters, output_size)

    def forward(self, x):
      # change to indexes, then to tensor
        z2 = []
        for ls in x:
          z1 = []
          for word in ls:
            z1.append(vocab[word])
          z2.append(z1)
        #print(z2)
        x = torch.tensor(z2)
        x = self.embedding(x)
        #print(x.shape)
        x = x.permute(0, 2, 1)  # Permute the dimensions for Conv1d (batch_size, embedding_dim, sequence_length)
        x = [F.relu(conv(x)) for conv in self.convs]
        #print(x[0].shape)
        x = [F.max_pool1d(conv, conv.size(2)).squeeze(2) for conv in x]
        x = torch.cat(x, 1)
        x = self.fc(x)
        return torch.sigmoid(x)

In [ ]:
# test forward 
#num_filters = 128
filter_sizes = [3, 4, 5]

model = CNNClassifier(vocab_size, 128, 128, filter_sizes, 14, vocab)
for batch in train_loader:
  text = batch['text']
  labels = batch['labels']  
  outputs = model(text)
  print(outputs.shape)
  break
print(model)

torch.Size([64, 14])
CNNClassifier(
  (embedding): Embedding(33824, 128)
  (convs): ModuleList(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    (1): Conv1d(128, 128, kernel_size=(4,), stride=(1,))
    (2): Conv1d(128, 128, kernel_size=(5,), stride=(1,))
  )
  (fc): Linear(in_features=384, out_features=14, bias=True)
)


In [ ]:
vocab_size = len(vocab)
embedding_dim = 100
num_filters = 100
filter_sizes = [3, 4, 5]
output_size = 14


In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
n_epochs = 5
model.train()
def train_model(model, train_dataloader, val_dataloader, epochs=n_epochs, optimizer=optimizer, criterion=criterion):
  #model.to(device)
  for epoch in range(epochs):
      curr_epoch_loss = []
      for batch in train_dataloader:
          text = batch['text']
          labels = batch['labels']
          optimizer.zero_grad()
          outputs = model(text)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          curr_epoch_loss.append(loss.cpu().data.numpy())
      print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss):.4f}")
      # Evaluation
      model.eval()
      val_loss = 0.0
      with torch.no_grad():
          for batch in val_dataloader:
              text = batch['text']
              labels = batch['labels']
              outputs = model(text)
              val_loss += criterion(outputs, labels).item()    
      avg_val_loss = val_loss / len(val_dataloader)
      print(f'Epoch [{epoch+1}/{epochs}], Val Loss: {avg_val_loss:.4f}')
  return model

In [ ]:
trained_model = train_model(model, train_loader, val_loader)

Epoch 0: curr_epoch_loss=0.4525
Epoch [1/5], Val Loss: 0.4315
Epoch 1: curr_epoch_loss=0.3405
Epoch [2/5], Val Loss: 0.3970
Epoch 2: curr_epoch_loss=0.3050
Epoch [3/5], Val Loss: 0.3964
Epoch 3: curr_epoch_loss=0.2802
Epoch [4/5], Val Loss: 0.3737
Epoch 4: curr_epoch_loss=0.2560
Epoch [5/5], Val Loss: 0.3756


In [ ]:
trained_model.type

<bound method Module.type of CNNClassifier(
  (embedding): Embedding(33824, 128)
  (convs): ModuleList(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    (1): Conv1d(128, 128, kernel_size=(4,), stride=(1,))
    (2): Conv1d(128, 128, kernel_size=(5,), stride=(1,))
  )
  (fc): Linear(in_features=384, out_features=14, bias=True)
)>

In [ ]:
def eval_model(model, dataloader):
    model.eval()
    Y_pred = []
    Y_true = []
    with torch.no_grad():
        for batch in dataloader:
            text = batch['text']
            labels = batch['labels']
            pred = model(text)
            pred=np.round(pred)
            Y_pred.append(pred)
            Y_true.append(labels)
    return Y_pred, Y_true

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
y_pred, y_true = eval_model(trained_model, val_loader)



In [ ]:
from sklearn.metrics import classification_report

target_names =["cohort","Obesity", "Non Adherence", "Developmental Delay Retardation", 
               "Advanced Heart Disease", "Advanced Lung Disease", "Schizophrenia and other Psychiatric Disorders",
               "Alcohol Abuse", "Other Substance Abuse", "Chronic Pain Fibromyalgia", 
               "Chronic Neurological Dystrophies", "Advanced Cancer", "Depression", "Dementia"]

Y_pred = np.vstack((y_pred[0],         y_pred[1],         y_pred[2]))
Y_true = np.vstack((y_true[0].numpy(), y_true[1].numpy(), y_true[2].numpy()))

print(Y_true.shape)
if Y_pred.all() == Y_true.all():
    print("The lists are identical")
else:
    print("The lists are not identical")
#classification_report(y_true[1].numpy(), y_pred[1], target_names=target_names, zero_division=1)
classification_report(Y_true, Y_pred, target_names=target_names, zero_division=1)


(167, 14)
The lists are identical


'                                               precision    recall  f1-score   support\n\n                                       cohort       1.00      0.00      0.00        12\n                                      Obesity       1.00      0.00      0.00        18\n                                Non Adherence       1.00      0.00      0.00         7\n              Developmental Delay Retardation       0.33      0.04      0.07        27\n                       Advanced Heart Disease       1.00      0.00      0.00        18\n                        Advanced Lung Disease       1.00      0.00      0.00        40\nSchizophrenia and other Psychiatric Disorders       1.00      0.00      0.00        27\n                                Alcohol Abuse       1.00      0.00      0.00        21\n                        Other Substance Abuse       1.00      0.00      0.00        35\n                    Chronic Pain Fibromyalgia       1.00      0.00      0.00        37\n             Chronic Neurolog

In [ ]:
roc_auc_score(y_true[0].numpy(), y_pred[0])

ValueError: ignored

                                             precision    recall  f1-score   support

                                       cohort       1.00      0.83      0.91         6
                                      Obesity       0.50      0.10      0.17        10
                                Non Adherence       1.00      1.00      1.00         1
              Developmental Delay Retardation       1.00      0.54      0.70        13
                       Advanced Heart Disease       0.00      0.00      0.00         7
                        Advanced Lung Disease       0.78      0.50      0.61        14            
                        Schizophrenia               0.86      0.55      0.67        11
                                Alcohol Abuse       0.86      0.60      0.71        10
                        Other Substance Abuse       1.00      0.46      0.63        13
                    Chronic Pain Fibromyalgia       1.00      0.42      0.59        12
             Chronic Neurological Dystrophies       1.00      0.67      0.80         3
                              Advanced Cancer       0.84      0.64      0.73        25
                                   Depression       1.00      0.25      0.40         4
                                     Dementia       0.00      0.00      0.00         8

# New Section

                                               precision    recall  f1-score   support

                                       cohort       1.00      0.00      0.00         4
                                      Obesity       0.00      0.00      0.00         3
                                Non Adherence       1.00      0.00      0.00         1
              Developmental Delay Retardation       1.00      0.31      0.47        13
                       Advanced Heart Disease       1.00      0.00      0.00         4
                        Advanced Lung Disease       0.00      0.00      0.00         9            
                        Schizophrenia               1.00      0.12      0.22         8
                                Alcohol Abuse       1.00      0.00      0.00         4
                        Other Substance Abuse       0.00      0.00      0.00        10
                    Chronic Pain Fibromyalgia       1.00      0.00      0.00        19
             Chronic Neurological Dystrophies       1.00      0.00      0.00         5
                              Advanced Cancer       1.00      0.78      0.88        18
                                   Depression       1.00      0.00      0.00         6
                                     Dementia       1.00      0.00      0.00         5

In [ ]:
                                               precision    recall  f1-score   support

                                       cohort       1.00      0.00      0.00         4
                                      Obesity       0.00      0.00      0.00         3
                                Non Adherence       1.00      0.00      0.00         1
              Developmental Delay Retardation       1.00      0.31      0.47        13
                       Advanced Heart Disease       1.00      0.00      0.00         4
                        Advanced Lung Disease       0.00      0.00      0.00         9
Schizophrenia and other Psychiatric Disorders       1.00      0.12      0.22         8
                                Alcohol Abuse       1.00      0.00      0.00         4
                        Other Substance Abuse       0.00      0.00      0.00        10
                    Chronic Pain Fibromyalgia       1.00      0.00      0.00        19
             Chronic Neurological Dystrophies       1.00      0.00      0.00         5
                              Advanced Cancer       1.00      0.78      0.88        18
                                   Depression       1.00      0.00      0.00         6
                                     Dementia       1.00      0.00      0.00         5
                                     